<a href="https://colab.research.google.com/github/daniel-nuno/time_series_O2024_MAF3074N/blob/main/evaluacion_pronosticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

![iteso](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

<br>
<br>
Clase: Series de tiempo
<br>
Actividad: Evaluación y Pronósticos
<br>
<br>

* * *

Docente: Daniel Nuño <br>
Fecha: 7 de octubre 2024 <br>

* * *

<br>
<br>

# Evaluación de pronósticos

Es importante evaluar la precisión de las previsiones utilizando previsiones auténticas. Por consiguiente, el tamaño de los residuos no es una indicación fiable de la magnitud de los errores de previsión reales. La precisión de las previsiones sólo puede determinarse teniendo en cuenta el rendimiento de un modelo con datos nuevos que no se utilizaron al ajustarlo.

Hay que tener en cuenta los siguientes puntos.

* Un modelo que se ajuste bien a los datos de entrenamiento no necesariamente pronosticará bien.
* Siempre se puede obtener un ajuste perfecto utilizando un modelo con suficientes parámetros.
* Ajustar demasiado un modelo a los datos es tan malo como no identificar un patrón sistemático en los datos.
* Algunas referencias describen el conjunto de prueba como el «conjunto de retención» porque estos datos se «retienen» de los datos utilizados para el ajuste.

Otras referencias llaman al conjunto de entrenamiento «datos dentro de la muestra» y al conjunto de prueba «datos fuera de la muestra». En este libro preferimos utilizar «datos de entrenamiento» y «datos de prueba».

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy as sp
import yfinance as yf

In [14]:
data = pd.read_excel("/content/drive/MyDrive/series_tiempo/Assets/remesas.xlsx")
data.index = pd.to_datetime(data['Fecha'])
del data['Fecha']
del data['Promedio Diario']
data = pd.Series(data = data['Remesas Totales'], index=data.index)

In [15]:
data.tail()

,Remesas Totales
Fecha,
2024-03-01,5014.5816
2024-04-01,5418.8526
2024-05-01,5618.1155
2024-06-01,6206.8793
2024-07-01,5613.6041


In [16]:
training_mask = data.index > '2023-01-01'

In [19]:
test = data[training_mask]
train = data[~training_mask]
horizon = len(test)

In [20]:
test.head()

,Remesas Totales
Fecha,
2023-02-01,4346.1517
2023-03-01,5189.3920
2023-04-01,5006.8015
2023-05-01,5675.8614
2023-06-01,5584.4913


In [22]:
pip install statsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
from statsforecast.models import SeasonalNaive

In [27]:
model_sn = SeasonalNaive(season_length=12)
model_sn = model_sn.fit(y=train.values)
y_hat_dict = model_sn.predict(h=horizon)

y_hat_dict['mean']

array([3928.4852, 4712.0937, 4749.1098, 5173.7153, 5150.2223, 5342.837 ,
       5160.1441, 5078.5862, 5404.6702, 4853.746 , 5379.1316, 4434.7719,
       3928.4852, 4712.0937, 4749.1098, 5173.7153, 5150.2223, 5342.837 ])

In [39]:
model_sn.predict_in_sample()

SeasonalNaive

In [50]:
residuals_train = train.values[12:] - model_sn.predict_in_sample()['fitted'][12:]

In [51]:
residuals_test = test.values - y_hat_dict['mean']

In [52]:
rmse_trian = np.sqrt(np.mean(residuals_train**2))
rmse_test = np.sqrt(np.mean(residuals_test**2))

In [53]:
tabla_modelos = pd.DataFrame(columns=['Modelo', 'RMSE Train', 'RMSE Test', 'Parametros'], data = [['Seasonal Naive', rmse_trian, rmse_test, 0]])

In [54]:
tabla_modelos

,Modelo,RMSE Train,RMSE Test,Parametros
0,Seasonal Naive,316.560422,467.073,0


## RMSE

En machine learning, el RMSE (Root Mean Squared Error o Raíz del Error Cuadrático Medio) es una métrica comúnmente utilizada para evaluar el rendimiento de un modelo en problemas de regresión. Estas son algunas de sus principales ventajas:

* Penaliza los errores grandes: Como el RMSE eleva al cuadrado las diferencias
entre las predicciones y los valores reales antes de promediar, los errores más grandes tienen un impacto mayor en la métrica final. Esto es útil cuando deseas que el modelo sea particularmente sensible a los errores grandes.
* Fácil interpretación: El RMSE está en la misma escala que la variable de salida, lo que facilita su interpretación. Por ejemplo, si estás prediciendo precios de casas en miles de dólares, un RMSE de 5 significa que el error promedio es de 5,000 dólares.
* Sensibilidad a cambios pequeños: Debido a la elevación al cuadrado, el RMSE puede detectar cambios pequeños en las predicciones, lo que lo hace adecuado cuando se busca mejorar el modelo finamente.
* Comparación entre modelos: RMSE es una métrica estándar ampliamente utilizada, lo que facilita la comparación del rendimiento de diferentes modelos de regresión en un conjunto de datos específico.
* Diferenciable: En el contexto del entrenamiento de modelos basados en métodos de optimización como el gradiente descendente, el RMSE es una función diferenciable, lo que permite calcular su gradiente y optimizar los pesos del modelo de manera eficiente.

## R-cuadrada

La principal diferencia entre RMSE (Root Mean Squared Error) y R-cuadrada (Coeficiente de Determinación) radica en el tipo de información que ofrecen y en cómo se interpretan en el contexto de los modelos de regresión.

1. RMSE (Root Mean Squared Error)
Métrica de error: El RMSE mide directamente el error promedio de las predicciones de un modelo. Cuantifica la magnitud del error en las predicciones del modelo, representando la distancia media entre los valores observados y los valores predichos.
Interpretación: Se interpreta en las mismas unidades que la variable dependiente o de salida. Un RMSE más bajo indica un modelo más preciso, pero no proporciona un sentido claro de qué tan bien el modelo está explicando la variabilidad de los datos.
Escala: Depende de las unidades de la variable dependiente. Esto significa que puede ser difícil comparar el RMSE entre modelos aplicados a diferentes conjuntos de datos o variables de diferentes escalas.
2. R-cuadrada (Coeficiente de Determinación)
* Métrica de ajuste: mide qué proporción de la variabilidad total en los datos de salida está explicada por el modelo. No mide el error en sí, sino qué tan bien las predicciones del modelo capturan las tendencias generales en los datos.
* Interpretación: va de 0 a 1 (o puede ser negativo si el modelo es peor que una línea de base), donde:
*   =1 significa que el modelo predice perfectamente todos los puntos.

*  =0 significa que el modelo no predice mejor que la media de los datos (es decir, no explica nada de la variabilidad).
*   < 0 significa que el modelo es peor que simplemente usar la media como predicción.

* Escala relativa: Como es una proporción, es más fácil de interpretar y comparar entre diferentes modelos o conjuntos de datos, ya que no está influenciado por las unidades de la variable dependiente.

3. Comparación clave:
* RMSE se centra en la magnitud del error en las predicciones. Si el RMSE es pequeño, significa que el modelo predice bien en términos absolutos.
* R-cuadrada se enfoca en la proporción de la variabilidad de los datos que el modelo puede explicar. Un alto R-cuadrada significa que el modelo está capturando bien las tendencias generales de los datos, pero no garantiza que los errores individuales sean pequeños (ahí es donde entra el RMSE).

**En resumen:**

* RMSE te dice "cuánto se están equivocando las predicciones en promedio.
* R-cuadrada "te dice "qué porcentaje de la variabilidad en los datos está siendo explicado por el modelo."

Ambas métricas son útiles, pero proporcionan información diferente. Es común usar ambas para evaluar modelos de regresión.

$$
R_{2} = 1 - \frac{SS_{res}}{SS_{tot}}
$$

$ SS_{res} $: es la suma de los cuadrados de los residuos (pronósticos) (también llamado error residual)

$ SS_{tot} $: es la suma total de los cuadrados (media) (también llamado total de la variabilidad):

In [36]:
pd.concat([tabla_modelos, pd.DataFrame(columns=['Modelo', 'RMSE', 'Parametros'], data = [['Regresion', rmse, 0]])])

,Modelo,RMSE,Parametros
0,Seasonal Naive,467.073,0
0,Regresion,467.073,0
